# Data analysis

This inefficient Notebook was only used to produce Figure 4.10 of the thesis.

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import openpmd_api as io
from scipy import constants, stats
from scipy.optimize import curve_fit
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
#    'font.family': 'sans-serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

from matplotlib.colors import LogNorm, SymLogNorm, Normalize, hsv_to_rgb, Normalize

In [ ]:
#path="/bigdata/hplsim/production/wrobel45/PWFA-bunch-bachelor1/particles_300MeV/040_particles_bachelor1/simOutput/openPMD/simData_beta_%T.bp"
#path_field="/bigdata/hplsim/production/wrobel45/PWFA-bunch-bachelor1/particles_300MeV/040_particles_bachelor1/simOutput/openPMD/simData_%T.bp"

path="/bigdata/hplsim/production/wrobel45/PWFA-bunch-bachelor1/particles_250MeV/034_particles_bachelor1/simOutput/openPMD/simData_beta_%T.bp"
path_field="/bigdata/hplsim/production/wrobel45/PWFA-bunch-bachelor1/particles_250MeV/034_particles_bachelor1/simOutput/openPMD/simData_%T.bp"

#path="/bigdata/hplsim/production/wrobel45/PWFA-bunch-bachelor1/downramp_250MeV/030_downramp_bachelor1/simOutput/openPMD/simData_beta_%T.bp"
#path_field="/bigdata/hplsim/production/wrobel45/PWFA-bunch-bachelor1/downramp_250MeV/030_downramp_bachelor1/simOutput/openPMD/simData_%T.bp"

#path="/bigdata/hplsim/production/wrobel45/PWFA-bunch-bachelor1/square_250MeV/030_square_bachelor1/simOutput/openPMD/simData_beta_%T.bp"
#path_field="/bigdata/hplsim/production/wrobel45/PWFA-bunch-bachelor1/square_250MeV/030_square_bachelor1/simOutput/openPMD/simData_%T.bp"


## Definitions

Define different functions for data fitting.

In [ ]:
def y_center(t, t_0, y_0):
    Delta_t = 1.706e-16/1.28631
    y_diff = y_0 + Delta_t * constants.c * (t-t_0) * 1e6

    return(y_diff)

In [ ]:
def window(series, t_0):
    """Returns start and end of viewing window in both directions and difference factor between y so driver is always centered at 0."""
    y_cells = 2048
    z_cells = 1024      # should be same as x_cells
    y_gpus = 8
    cellwidth = 0.5 * 0.1772e-6   # same for all directions
    Delta_t = 1.706e-16/1.28631
    windowwidth = y_cells * (y_gpus-1)/y_gpus * cellwidth * 1e6 # in \mu m
    windowheight = z_cells * cellwidth * 1e6 # in \mu m
    
    par_beg = particlePMD(series.iterations[t_0].particles["b_all"], series)
    y_0 = np.mean(par_beg.y_pos)
        
    y_min = -windowwidth * 2.4/4 *0.2
    y_max = windowwidth * 1.6/4 *0.2
    z_min = -windowheight / 2
    z_max =  windowheight / 2
        
    return y_min, y_max, z_min, z_max, y_0

In [ ]:
def time_to_dist(time, array=False):
    Delta_t = 1.706e-16/1.28631
    cellwidth = 0.5 * 0.1772e-6   # same for all directions
    start_pos = (10191 * 1.02) * Delta_t * constants.c + 2048 * cellwidth
    
    t_min = time[0]
    t_max = time[-1]
    t_diff = (t_max-t_min) / (len(time)-1)
    d_min = ((t_min - t_diff/2) * Delta_t * constants.c - start_pos) * 1e3
    d_max = ((t_max + t_diff/2) * Delta_t * constants.c - start_pos) * 1e3
    d_array = (time * Delta_t * constants.c - start_pos) * 1e3
    if array:
        return d_min, d_max, d_array
    else:
        return d_min, d_max

In [ ]:
def Gauss(x, a, b, c):
    y = a * np.exp(-(x-b)**2 / (2*c**2))
    return y

def Gauss_dens(x, a, b):
    y = 1/np.sqrt(2*np.pi * b**2) * np.exp(-(x-a)**2 / (2*b**2))
    return y

def Gauss2(x, a1, b1, c1, a2, b2, c2):
    y = a1 * np.exp(-(x-b1)**2 / (2*c1**2)) + a2 * np.exp(-(x-b2)**2 / (2*c2**2))
    return y

def Gauss3(x, a1, b1, c1):
    y = c1 * np.exp(-(x-a1)**2 / (2*b1**2))
    return y

def Lorentz(x, a, b):
    y = 2/np.pi * b / (4*(x - a)**2 + b)
    return y

def Max_Boltz_2D(x, a):
    y = x/a * np.exp(-x**2 / (2*a))
    return y

In [ ]:
class particlePMD:
    def __init__(self, particle, series_par):
        x_off = particle["positionOffset"]["x"][:]
        x_off_unit = particle["positionOffset"]["x"].unit_SI
        y_off = particle["positionOffset"]["y"][:]
        y_off_unit = particle["positionOffset"]["y"].unit_SI
        z_off = particle["positionOffset"]["z"][:]
        z_off_unit = particle["positionOffset"]["z"].unit_SI

        x = particle["position"]["x"][:]
        x_unit = particle["position"]["x"].unit_SI
        y = particle["position"]["y"][:]
        y_unit = particle["position"]["y"].unit_SI
        z = particle["position"]["z"][:]
        z_unit = particle["position"]["z"].unit_SI

        p_x = particle["momentum"]["x"][:]
        p_x_unit = particle["momentum"]["x"].unit_SI
        p_y = particle["momentum"]["y"][:]
        p_y_unit = particle["momentum"]["y"].unit_SI
        p_z = particle["momentum"]["z"][:]
        p_z_unit = particle["momentum"]["z"].unit_SI

        x_probeE = particle['probeE']['x'][:]
        x_probeE_unit = particle["probeE"]["x"].unit_SI 
        y_probeE = particle['probeE']['y'][:]
        y_probeE_unit = particle["probeE"]["y"].unit_SI
        z_probeE = particle['probeE']['z'][:]
        z_probeE_unit = particle["probeE"]["z"].unit_SI

        x_probeB = particle['probeB']['x'][:]
        x_probeB_unit = particle["probeB"]["x"].unit_SI 
        y_probeB = particle['probeB']['y'][:]
        y_probeB_unit = particle["probeB"]["y"].unit_SI
        z_probeB = particle['probeB']['z'][:]
        z_probeB_unit = particle["probeB"]["z"].unit_SI

        particleId = particle["id"][io.Mesh_Record_Component.SCALAR][:]
        weight = particle["weighting"][io.Mesh_Record_Component.SCALAR][:]

        series_par.flush()

        self.x_pos = (x * x_unit + (x_off-512) * x_off_unit) * 1e6
        self.y_pos = (y * y_unit +  y_off      * y_off_unit) * 1e6
        self.z_pos = (z * z_unit + (z_off-512) * z_off_unit) * 1e6
        
        self.x_mom = p_x * p_x_unit
        self.y_mom = p_y * p_y_unit
        self.z_mom = p_z * p_z_unit
        
        self.x_probeE = x_probeE * x_probeE_unit
        self.y_probeE = y_probeE * y_probeE_unit
        self.z_probeE = z_probeE * z_probeE_unit
        
        self.x_probeB = x_probeB * x_probeB_unit
        self.y_probeB = y_probeB * y_probeB_unit
        self.z_probeB = z_probeB * z_probeB_unit
        
        self.particleId = particleId
        self.weight = np.float64(weight)
        
    def Energy(self):
        # squared total momentum
        p2_total = np.float64(self.x_mom**2 + self.y_mom**2 + self.z_mom**2)
        # energy per macro particle in Joule
        E_J = (constants.c * np.sqrt((constants.m_e*self.weight)**2 * constants.c**2 + p2_total) - (constants.m_e*self.weight) * constants.c**2) * 1/self.weight
        # conversion to MeV
        E = E_J / constants.e / 1e6
        
        return E
        
    def LorentzForce(self):
        x_vel = np.sqrt((self.x_mom / (constants.m_e*self.weight))**2 / (1 + (self.x_mom**2 + self.y_mom**2 + self.z_mom**2) / (constants.m_e*self.weight)**2 / constants.c**2))
        y_vel = np.sqrt((self.y_mom / (constants.m_e*self.weight))**2 / (1 + (self.x_mom**2 + self.y_mom**2 + self.z_mom**2) / (constants.m_e*self.weight)**2 / constants.c**2))
        z_vel = np.sqrt((self.z_mom / (constants.m_e*self.weight))**2 / (1 + (self.x_mom**2 + self.y_mom**2 + self.z_mom**2) / (constants.m_e*self.weight)**2 / constants.c**2))

        F_x_B = y_vel*self.z_probeB - z_vel*self.y_probeB
        F_y_B = z_vel*self.x_probeB - x_vel*self.z_probeB
        F_z_B = x_vel*self.y_probeB - y_vel*self.x_probeB

        F_x_L = -constants.e*self.weight * (self.x_probeE + F_x_B) * 1e3
        F_y_L = -constants.e*self.weight * (self.y_probeE + F_y_B) * 1e3
        F_z_L = -constants.e*self.weight * (self.z_probeE + F_z_B) * 1e3   
        
        return F_x_L, F_y_L, F_z_L

In [ ]:
def binned_to_grid(bin_x, bin_y):
    """Transforms 2D bins into grid.
    The left borders of the bins are taken as gridpoints, the rightmost border is discarded.
    """
    x_width = bin_x[1] - bin_x[0]
    y_width = bin_y[1] - bin_y[0]
    x = bin_x[:-1] + x_width/2
    y = bin_y[:-1] + y_width/2
    
    return x, y

## Density fields

Plots the charge-, energy-density and E-fields for b- and e-Particles

In [ ]:
# Test for density plots
time = np.arange(12000, 170000+1, 4000)
series = io.Series(path_field, io.Access.read_only)
cellwidth = 0.5 * 0.1772e-6 * 1e6   # in micorometer, same for all directions

y_start = 700
y_end = 1300
z_start = 256
z_end = 512+z_start

z_min = -(z_start-0.5)*cellwidth
z_max = (z_start-0.5)*cellwidth
y_min = (y_start-0.5)*cellwidth-100
y_max = (y_end-0.5)*cellwidth-100

y = np.arange(y_start, y_end)*cellwidth-100
z = np.arange(-z_start, z_start)*cellwidth

e_dens_hist = np.zeros((len(time), y_end-y_start))
E_field_hist = np.zeros((len(time), y_end-y_start))

for i, t in enumerate(time):
    
    e_field_charge = series.iterations[t].meshes["e_all_chargeDensity"][io.Mesh_Record_Component.SCALAR][z_start:z_end,y_start:y_end,512]
    e_field_charge_unit = series.iterations[t].meshes["e_all_chargeDensity"][io.Mesh_Record_Component.SCALAR].unit_SI
    e_field_energy = series.iterations[t].meshes["e_all_energyDensity"][io.Mesh_Record_Component.SCALAR][z_start:z_end,y_start:y_end,512]
    e_field_energy_unit = series.iterations[t].meshes["e_all_energyDensity"][io.Mesh_Record_Component.SCALAR].unit_SI
    b_field_charge = series.iterations[t].meshes["b_all_chargeDensity"][io.Mesh_Record_Component.SCALAR][z_start:z_end,y_start:y_end,512]
    b_field_charge_unit = series.iterations[t].meshes["b_all_chargeDensity"][io.Mesh_Record_Component.SCALAR].unit_SI
    b_field_energy = series.iterations[t].meshes["b_all_energyDensity"][io.Mesh_Record_Component.SCALAR][z_start:z_end,y_start:y_end,512]
    b_field_energy_unit = series.iterations[t].meshes["b_all_energyDensity"][io.Mesh_Record_Component.SCALAR].unit_SI
    E_field_x = series.iterations[t].meshes["E"]["x"][z_start:z_end,y_start:y_end,512]
    E_field_x_unit = series.iterations[t].meshes["E"]["x"].unit_SI
    E_field_y = series.iterations[t].meshes["E"]["y"][z_start:z_end,y_start:y_end,512]
    E_field_y_unit = series.iterations[t].meshes["E"]["y"].unit_SI
    E_field_z = series.iterations[t].meshes["E"]["z"][z_start:z_end,y_start:y_end,512]
    E_field_z_unit = series.iterations[t].meshes["E"]["z"].unit_SI
    
    series.flush()
    
    # unit conversion
    # charge in pC/um^2 = C/m^2
    e_field_charge *= e_field_charge_unit*1e-6
    b_field_charge *= b_field_charge_unit*1e-6
    e_field_energy *= e_field_energy_unit/constants.e*1e-12
    b_field_energy *= b_field_energy_unit/constants.e*1e-12
    E_field_x *= E_field_x_unit*1e-9
    E_field_y *= E_field_y_unit*1e-9
    E_field_z *= E_field_z_unit*1e-9
    abs_Eyz = np.sqrt(E_field_x**2 + E_field_y**2 + E_field_z**2)

    print("E_field: "       , np.min(E_field_y)     , np.max(E_field_y))
    print("e_field_charge: ", np.min(e_field_charge), np.max(e_field_charge))
    print("e_field_energy: ", np.min(e_field_energy), np.max(e_field_energy))
    print("b_field_charge: ", np.min(b_field_charge), np.max(b_field_charge))
    print("b_field_energy: ", np.min(b_field_energy), np.max(b_field_energy))
    
    e_dens_hist[i] = e_field_charge[int((z_end-z_start)/2),:]
    E_field_hist[i] = E_field_y[int((z_end-z_start)/2),:]
    
    plot = False
    if plot:
        plt.figure(figsize=(16, 12), dpi=70)

        plt.subplot(221)
        plt.imshow(-e_field_charge, cmap='hot_r', norm=LogNorm(vmin=1e-3, vmax=2e2), extent=(y_min, y_max, z_min, z_max), aspect='auto', origin="lower", interpolation='none')
        plt.colorbar(label="$dQ_{e^-}/dx/dy/dz \, \mathrm{[pC/\mu m^3]}$")    
        plt.imshow(-b_field_charge, cmap='Greens', norm=LogNorm(vmin=1e-3, vmax=1e1), extent=(y_min, y_max, z_min, z_max), aspect='auto', origin="lower", interpolation='none')
        plt.colorbar(label="$dQ_{bunch}/dx/dy/dz \, \mathrm{[pC/\mu m^3]}$") 
        plt.xlabel("y")
        plt.ylabel("z")
        plt.title("charge density, timestep = {}".format(t))

        plt.subplot(222)
        plt.imshow(e_field_energy, cmap='hot_r', norm=LogNorm(vmin=1e-9, vmax=1e2), extent=(y_min, y_max, z_min, z_max), aspect='auto', origin="lower", interpolation='none')
        plt.colorbar(label="$dE_{e^-}/dx/dy/dz \, \mathrm{[MeV/\mu m^3]}$")    
        plt.imshow(b_field_energy, cmap='Greens', norm=LogNorm(vmin=1e-3, vmax=1e3), extent=(y_min, y_max, z_min, z_max), aspect='auto', origin="lower", interpolation='none')
        plt.colorbar(label="$dE_{bunch}/dx/dy/dz \, \mathrm{[MeV/\mu m^3]}$")    
        plt.xlabel("$y \, \mathrm{[\mu m]}$")
        plt.ylabel("$z \, \mathrm{[\mu m]}$")
        plt.title("energy density, timestep = {}".format(t))

        plt.subplot(223)
        plt.imshow(E_field_y, cmap='Spectral', vmin=-2e2, vmax=2e2, extent=(y_min, y_max, z_min, z_max), aspect='auto', origin="lower", interpolation='none')
        plt.colorbar(label="$E_y \, \mathrm{[GeV/m]}$")    
        plt.streamplot(y, z, E_field_y, E_field_z, color=abs_Eyz, cmap='magma_r', density=2, norm=Normalize(vmin=0, vmax=3e2))
        plt.colorbar(label="$\left|\\vec{E}\\right| \, \mathrm{[GeV/m]}$")    
        plt.imshow(-b_field_charge, cmap='Greens', norm=LogNorm(vmin=1e-5, vmax=2e0), extent=(y_min, y_max, z_min, z_max), aspect='auto', origin="lower", interpolation='none')
        plt.colorbar(label="charge density") 
        plt.xlabel("$y \, \mathrm{[\mu m]}$")
        plt.ylabel("$z \, \mathrm{[\mu m]}$")
        plt.title("E-field, timestep = {}".format(t))
        
        plt.show()
        #plt.savefig(fname="temp", facecolor="white")

del series


# Plots for thesis

## fig: cavity_low

In [ ]:
# Test for density plots
t = 96000
series = io.Series(path_field, io.Access.read_only)
Delta_t = 1.706e-16/1.28631
cellwidth = 0.5 * 0.1772e-6   # same for all directions
start_pos = (10191 * 1.02) * Delta_t * constants.c + 2048 * cellwidth

d_min, dmax = time_to_dist(time)
d_array = (t * Delta_t * constants.c - start_pos) * 1e3
y_start = 600
y_end = 1300
z_start = 256
z_end = 512+z_start

cellwidth *= 1e6
z_min = -(z_start-0.5)*cellwidth
z_max = (z_start-0.5)*cellwidth
y_min = (y_start-0.5)*cellwidth-100
y_max = (y_end-0.5)*cellwidth-100

y = np.arange(y_start, y_end)*cellwidth-100
z = np.arange(-z_start, z_start)*cellwidth

fig = plt.figure(figsize=(6.4, 6))
ax1, ax2 = fig.subplots(2, 1, sharex=True)
cax1 = fig.add_axes([0.83, 0.15, 0.03, 0.3])
cax2 = fig.add_axes([0.83, 0.56, 0.03, 0.3])
    
e_field_charge = series.iterations[t].meshes["e_all_chargeDensity"][io.Mesh_Record_Component.SCALAR][z_start:z_end,y_start:y_end,512]
e_field_charge_unit = series.iterations[t].meshes["e_all_chargeDensity"][io.Mesh_Record_Component.SCALAR].unit_SI
b_field_charge = series.iterations[t].meshes["b_all_chargeDensity"][io.Mesh_Record_Component.SCALAR][z_start:z_end,y_start:y_end,512]
b_field_charge_unit = series.iterations[t].meshes["b_all_chargeDensity"][io.Mesh_Record_Component.SCALAR].unit_SI
    
series.flush()
del series

# unit conversion
# charge in pC/um^3 = 1e-6 C/m^3
e_field_charge *= e_field_charge_unit*1e-6
b_field_charge *= b_field_charge_unit*1e-6
    
im = ax1.imshow(-e_field_charge, cmap='hot_r', norm=LogNorm(vmin=1e-3, vmax=2e2), extent=(y_min, y_max, z_min, z_max), aspect='auto', origin="lower", interpolation='none')
im2 = ax1.imshow(-b_field_charge, cmap='Greens', norm=LogNorm(vmin=1e-3, vmax=1e1), extent=(y_min, y_max, z_min, z_max), aspect='auto', origin="lower", interpolation='none')
ax1.set_ylabel("$z \, \mathrm{[\mu m]}$")

ax1.set_title("a)")

im3 = ax2.imshow(-e_dens_hist, cmap="hot_r", origin="lower", aspect='auto', norm=LogNorm(vmin=1e-3, vmax=2e2), extent=(y_min, y_max, d_min, dmax), interpolation='none')
ax2.set_xlabel("$\zeta \, \mathrm{[\mu m]}$")
ax2.set_ylabel("$y \,[mm]$")

ax2.set_title("b)") 

cbar1 = fig.colorbar(im, label="$-dQ_{e^-}/dx/dy/dz \, \mathrm{[pC/\mu m^3]}$", cax=cax1)
cbar2 = fig.colorbar(im2, label="$-dQ_{driver}/dx/dy/dz \, \mathrm{[pC/\mu m^3]}$", cax=cax2)
print(d_array)

fig.subplots_adjust(hspace=0.2, right=0.8)
#plt.show()
#plt.savefig(fname="../images/1temp.png", facecolor='white', dpi=100)
plt.savefig(fname="../images/cavity_low.pgf")
